In [2]:
!pip install redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.2/354.2 kB 870.5 kB/s eta 0:00:00a 0:00:01


In [3]:
import redis 


In [4]:
r = redis.Redis(
    host='redis',
    port=6379,
    db=0,
    decode_responses=True)
r.ping()

ConnectionError: Error -5 connecting to redis:6379. No address associated with hostname.

In [ ]:
def add_player(id,name,country,games_played, score):
    player_data = {     
        "name" : name,
        "country" : country,
        "games_played" : 0,
        "score" : score,
    }
    
    player_key = f"player:{id}"
        
    r.hset(player_key,mapping=player_data)

    leaderboard_key = "leaderboard"
    r.zadd(leaderboard_key,{id : score})

In [ ]:
add_player(1,"jorge","españa",13,1000)
add_player(1,"jorge","españa",13,1000)
add_player(2,"Pepe","españa",13,1000)

In [ ]:
def update_score(id,points):
    player_key = f"player:{id}"
    player_data = r.hgetall(player_key)
    player_data["score"] = int(player_data["score"]) + points
    player_data["games_played"] = int(player_data["games_played"]) + 1
    r.hset(player_key,mapping=player_data)

    r.zadd("leaderboard",{id:int(player_data["score"])})

In [ ]:
update_score("2",39000)

In [ ]:
def player_info(id):
    player_key = f"player:{id}"

    info = r.hgetall(player_key)
    
    for key, value in info.items():
        print(key +" > "+value)
       

In [ ]:
player_info(1)

name > jorge
country > españa
games_played > 0
score > 1000


In [ ]:
def show_top_players(n):
    top_players = r.zrevrange("leaderboard", 0, n -1, withscores=True)

    if not top_players:
        print("El ranking está vacio")
        return
    
    for i ,(id,score) in enumerate(top_players):
        player_key = f"player:{id}"

        res = r.hgetall(player_key)

        print(f"#{i+1} {res['name']} ({res['country']})")
        print(f"ID: {id} Score: {score}")

In [ ]:
show_top_players(2)

#1 Pepe (españa)
ID: 2 Score: 40000.0
#2 jorge (españa)
ID: 1 Score: 1000.0


In [ ]:
import datetime as date

def get_date_key(date_obj):
    return f"unique:players:{date_obj.isoformat()}"

def register_login(id):

    today_date = date.date.today()
    
    today_key = get_date_key(today_date)
    #today_key = f"unique.:players:{today_key.isoformat()}"

    r.pfadd(today_key,id)

In [ ]:
register_login(2)

In [ ]:
def count_unique_logins(date_str):
    key = f"unique:players:{date_str}"
    count = r.pfcount(key)


    print(f"Jugadores unicos el {date_str} : {count}")

In [ ]:
count_unique_logins("2025-11-11")

Jugadores unicos el 2025-11-11 : 2


In [ ]:
def weekly_report(dates_list):
    keys_to_merge = [f"unique:players:{d}" for d in dates_list]

    report_key = "unique:players:week_report"
    r.pfmerge(report_key, *keys_to_merge)

    total_unique = r.pfcount(report_key)
    
    print(f"Reporte semanal (Dias: {len(dates_list)})")
    print(f"Total de jugadores únicos (aprox): {total_unique}")

    r.delete(report_key)
    return total_unique

In [ ]:
weekly_report(["2025-11-10", "2025-11-11"])

Reporte semanal (Dias: 2)
Total de jugadores únicos (aprox): 2


2